In [1]:
import tensorflow as tf
import os
import argparse
from tensorflow.python.keras.callbacks import Callback

from datetime import datetime, timezone
import logging

In [2]:
class MyFashionMnist(object):
  def train(self):
    
    # 입력 값을 받게 추가합니다.
    parser = argparse.ArgumentParser()
    parser.add_argument('--learning_rate', required=False, type=float, default=0.01)
    parser.add_argument('--dropout_rate', required=False, type=float, default=0.2)
    parser.add_argument('--opt', required=False, type=int, default=1)
    
    args = parser.parse_args()
    
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(args.dropout_rate),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.summary()
    
    sgd = tf.keras.optimizers.SGD(learning_rate=args.learning_rate)
    adam = tf.keras.optimizers.Adam(learning_rate=args.learning_rate)
    
    optimizers = [sgd, adam]
    model.compile(optimizer=optimizers[args.opt],
                  loss='sparse_categorical_crossentropy',
                  metrics=['acc'])

    model.fit(x_train, y_train,
              verbose=0,
              validation_data=(x_test, y_test),
              epochs=5,
              callbacks=[KatibMetricLog()])

    model.evaluate(x_test,  y_test, verbose=2)

class KatibMetricLog(Callback):
    def on_batch_end(self, batch, logs={}):
        print("batch=" + str(batch),
              "accuracy=" + str(logs.get('acc')),
              "loss=" + str(logs.get('loss')))
    def on_epoch_begin(self, epoch, logs={}):
        print("epoch " + str(epoch) + ":")
    
    def on_epoch_end(self, epoch, logs={}):
        # RFC 3339
        local_time = datetime.now(timezone.utc).astimezone().isoformat()
        logging.info(( "\n{} accuracy={:.4f} loss={:.4f} Validation-accuracy={:.4f} Validation-loss={:.4f}"
                       .format( local_time, logs.get('acc'), logs.get('loss'), 
                                logs.get('val_acc'), logs.get('val_loss') )
                      ))
        print("Validation-accuracy=" + str(logs.get('val_acc')),
              "Validation-loss=" + str(logs.get('val_loss')))
        return

def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")
    
    
if __name__ == '__main__':
    
    # For metricsCollector
    file_path = '/var/log/katib'
    createDirectory(file_path)
    logging.basicConfig(filename=file_path+'/'+'metrics.log', level=logging.DEBUG)
    
    if os.getenv('FAIRING_RUNTIME', None) is None:
        from kubeflow import fairing
        from kubeflow.fairing.kubernetes import utils as k8s_utils

        DOCKER_REGISTRY = '192.168.0.180:5000' #'kubeflow-registry.default.svc.cluster.local:30000'
        fairing.config.set_builder(
            'append',
            image_name='fairing-job-v1',
            base_image='brightfly/kubeflow-jupyter-lab:tf2.0-cpu',
            registry=DOCKER_REGISTRY, 
            push=True)
        # cpu 2, memory 5GiB
        fairing.config.set_deployer('job',
                                    namespace='space-openess', #'dudaji',
                                    pod_spec_mutators=[
                                        k8s_utils.get_resource_mutator(cpu=2,
                                                                       memory=5)]
         
                                   )
        fairing.config.run()
    else:
        remote_train = MyFashionMnist()
        remote_train.train()

4423680/4422102 [==============================] - 0s 0us/step
2022-09-30 05:51:31.051655: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2022-09-30 05:51:31.080569: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2400000000 Hz
2022-09-30 05:51:31.082100: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x4c86500 executing computations on platform Host. Devices:
2022-09-30 05:51:31.082145: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #
flatten (Flatten)            (None, 784)               0
_________________________________________________________________
dense (Dense)                (None, 128)               100480
___________________________